In [1]:
import pandas as pd

In [2]:
import requests

url = "https://realtor.p.rapidapi.com/properties/v2/list-for-rent"

beds_min = 0
price_max = 5000
price_min = 800
# prop_type = "single_family"
baths_min = 0
radius = 2
city = "Denver"
state_code = "CO"
zip_code = 80220

querystring = {"beds_min":beds_min,
               "price_max":price_max,
#                "prop_type":prop_type,
               "sort":"relevance",
               "baths_min":baths_min,
               "radius":radius,
               "price_min":price_min,
               "city":city,
               "state_code":state_code,
               "limit":"200",
               "postal_code": zip_code,
               "offset":"0"}

headers = {
    'x-rapidapi-host': "realtor.p.rapidapi.com",
    'x-rapidapi-key': "bdf6bde97amsh2f0d8da5849d952p1e4163jsn48237c3a4f33"
    }

response = requests.request("GET", url, headers=headers, params=querystring)
data = response.json()


In [3]:
rental_homes = data["properties"]
rental_homes[0]

{'property_id': 'R1650688430',
 'listing_id': '598431704',
 'prop_type': 'apartment',
 'list_date': '2016-08-05T15:32:00.000Z',
 'last_update': '2020-05-12T08:21:00.000Z',
 'year_built': 1965,
 'listing_status': 'active',
 'beds': 0,
 'prop_status': 'for_rent',
 'address': {'city': 'Denver',
  'country': 'USA',
  'county': 'Denver',
  'lat': 39.737231,
  'line': '1325 Garfield St',
  'postal_code': '80206',
  'state_code': 'CO',
  'state': 'Colorado',
  'time_zone': 'America/Denver',
  'neighborhood_name': 'Capital Avenue',
  'neighborhoods': [{'id': '4a8eefe0-7364-5964-ad8f-a2e495ab46e7',
    'level': 'sub_neighborhood',
    'name': 'Capital Avenue'},
   {'id': '98c26d70-5c4f-5a6a-9e75-842aa231f1c7',
    'level': 'neighborhood',
    'name': 'Congress Park'},
   {'id': '15555d80-9a49-5146-b9f0-c4cf9547bbcd',
    'level': 'macro_neighborhood',
    'name': 'Central Denver'}],
  'lon': -104.944294},
 'client_display_flags': {'presentation_status': 'for_rent',
  'is_showcase': True,
  'lea

In [4]:
rental_homes[0]

{'property_id': 'R1650688430',
 'listing_id': '598431704',
 'prop_type': 'apartment',
 'list_date': '2016-08-05T15:32:00.000Z',
 'last_update': '2020-05-12T08:21:00.000Z',
 'year_built': 1965,
 'listing_status': 'active',
 'beds': 0,
 'prop_status': 'for_rent',
 'address': {'city': 'Denver',
  'country': 'USA',
  'county': 'Denver',
  'lat': 39.737231,
  'line': '1325 Garfield St',
  'postal_code': '80206',
  'state_code': 'CO',
  'state': 'Colorado',
  'time_zone': 'America/Denver',
  'neighborhood_name': 'Capital Avenue',
  'neighborhoods': [{'id': '4a8eefe0-7364-5964-ad8f-a2e495ab46e7',
    'level': 'sub_neighborhood',
    'name': 'Capital Avenue'},
   {'id': '98c26d70-5c4f-5a6a-9e75-842aa231f1c7',
    'level': 'neighborhood',
    'name': 'Congress Park'},
   {'id': '15555d80-9a49-5146-b9f0-c4cf9547bbcd',
    'level': 'macro_neighborhood',
    'name': 'Central Denver'}],
  'lon': -104.944294},
 'client_display_flags': {'presentation_status': 'for_rent',
  'is_showcase': True,
  'lea

In [5]:
neighborhood = rental_homes[195]
neighborhood

{'property_id': 'R9492103587',
 'listing_id': '618205408',
 'prop_type': 'apartment',
 'last_update': '2020-04-24T23:28:42.000Z',
 'listing_status': 'active',
 'beds': 1,
 'baths_full': 1,
 'price_reduced_date': '2019-09-19T23:10:28.493Z',
 'prop_status': 'for_rent',
 'address': {'city': 'Denver',
  'country': 'USA',
  'county': 'Denver',
  'lat': 39.736022,
  'line': '8805 E 12th Ave Unit 1ST',
  'postal_code': '80220',
  'state_code': 'CO',
  'state': 'Colorado',
  'time_zone': 'America/Denver',
  'neighborhood_name': 'East Colfax',
  'neighborhoods': [{'id': 'b1409d08-6f4b-5245-aa86-d24c766430d3',
    'level': 'neighborhood',
    'name': 'East Colfax'},
   {'id': '1791b920-fb64-5517-92dd-c814c5aae28e',
    'level': 'macro_neighborhood',
    'name': 'East Denver'}],
  'lon': -104.885851},
 'client_display_flags': {'presentation_status': 'for_rent',
  'is_showcase': False,
  'lead_form_phone_required': False,
  'price_change': 0,
  'has_specials': False,
  'is_mls_rental': False,
  'i

In [6]:
from math import sin, cos, sqrt, atan2, radians
counter = 0
address_list = []
beds_min_list = []
beds_max_list = []
baths_min_list = []
baths_max_list = []
lat_list = []
lon_list =[]
distance_list = []
price_min_list = []
price_max_list = []
sqft_min_list = []
sqft_max_list = []
neighborhood_list = []
# photos_array = {}

for listing in rental_homes:
#     try:
        try:
            address = rental_homes[counter]["address"]["line"]
            
        except:
            address = "N/A"
        address_list.append(address)
        try:
            beds_min = rental_homes[counter]["community"]["beds_min"]      
            beds_max = rental_homes[counter]["community"]["beds_max"]
        except:
            beds_min = rental_homes[counter]["beds"]
            beds_max = rental_homes[counter]["beds"]
        
        
        beds_min_list.append(beds_min)
        beds_max_list.append(beds_max)
        
        try:
            try:
                sqft_min = rental_homes[counter]["community"]["sqft_min"]      
                sqft_max = rental_homes[counter]["community"]["sqft_max"]
            except:
                sqft_min = rental_homes[counter]["sqft"]
                sqft_max = rental_homes[counter]["sqft"]

        except:
            sqft_min = "N/A"
            sqft_max = "N/A"
#             print(address)
#             print(counter)
        sqft_min_list.append(sqft_min)
        sqft_max_list.append(sqft_max)
        
        try:
            try:
                baths_min = rental_homes[counter]["community"]["baths_min"]      
                baths_max = rental_homes[counter]["community"]["baths_max"]
            except:
                baths_min = rental_homes[counter]["baths"]
                baths_max = rental_homes[counter]["baths"]
        except:
            baths_min = beds_min
            baths_max = beds_max
    #             print(counter)
            
        baths_min_list.append(baths_min)
        baths_max_list.append(baths_max)
        
        
        lat = rental_homes[counter]["address"]["lat"]
        lon = rental_homes[counter]["address"]["lon"]
        try:
            neighborhood = rental_homes[counter]["address"]["neighborhood_name"]
        except:
            neighborhood = "Unknown"
        neighborhood_list.append(neighborhood)
        
        try:
            price_min = rental_homes[counter]["price"]
            price_max = rental_homes[counter]["price"]
        except(KeyError):
            price_min = rental_homes[counter]["community"]["price_min"]      
            price_max = rental_homes[counter]["community"]["price_max"]
        lat_list.append(lat)
        lon_list.append(lon)
        
        price_min_list.append(price_min)
        price_max_list.append(price_max)
#         photos_list = rental_homes[counter]["photos"][0]
        R = 6373.0
        lat1 = radians(39.7392)
        lon1 = radians(-39.73915 -104.9847)
        lat2 = radians(lat)
        lon2 = radians(lon)
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        distance_list.append(distance)
        counter = counter+1
#     except (KeyError, IndexError):
#         price_min = rental_homes[counter]["price"]
#         price_max = rental_homes[counter]["price"]

    


In [7]:
df = pd.DataFrame()

In [8]:
df["Address"] = address_list
df["Beds_Min"] = beds_min_list
df["Beds_Max"] = beds_max_list
df["Baths_Min"] = baths_min_list
df["Baths_Max"] = baths_max_list
df["Sqft_Min"] = sqft_min_list
df["Sqft_Max"] = sqft_max_list
df["Lat"] = lat_list
df["Lon"] = lon_list
df["Distance"] = distance_list
df["Rent"] = price_min_list
df["Rent_max"] = price_max_list
df["Neighborhood"] = neighborhood_list
df = df.dropna()
# df["Photos"] = photos_list

In [9]:
df.to_csv('excelview.csv')

In [10]:
df.sort_values(["Distance"])


,Address,Beds_Min,Beds_Max,Baths_Min,Baths_Max,Sqft_Min,Sqft_Max,Lat,Lon,Distance,Rent,Rent_max,Neighborhood
90,1511 E 22nd Avenue 22nd Ave,3,3,1.0,1.0,1800,1800,39.749550,-104.969226,3371.401741,2100,2100,City Park West
65,2150-2170 Franklin St,2,2,1.0,1.0,650,650,39.749280,-104.968060,3371.505656,1275,1275,City Park West
84,3331 N Race St,2,2,1.5,1.5,950,950,39.763875,-104.963555,3371.516206,2195,2195,Cole
33,1570-1578 Humboldt St,0,1,1.0,1.0,291,588,39.741330,-104.969330,3371.598813,1110,1695,City Park West
98,1691 N Franklin St Apt 306,0,0,1.0,1.0,N/A,N/A,39.742949,-104.968601,3371.619023,1319,1319,City Park West
...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,1588 Lima St Apt 206,1,1,1.0,1.0,N/A,N/A,39.741822,-104.856253,3381.008780,913,913,Boston Heights
168,11310 E 16th Ave Apt 307,2,2,1.0,1.0,N/A,N/A,39.741827,-104.855978,3381.031567,1219,1219,Boston Heights
167,11310 E 16th Ave Apt 106,1,1,1.0,1.0,N/A,N/A,39.741827,-104.855978,3381.031567,964,964,Boston Heights
137,1381 Lima St,5,5,2.0,2.0,N/A,N/A,39.738006,-104.856778,3381.060893,2250,2250,Aurora Heights


In [27]:
from math import sin, cos, sqrt, atan2, radians


    

# approximate radius of earth in km
R = 6373.0

lat1 = radians(39.663750)
lon1 = radians(-105.053680)
lat2 = radians(52.406374)
lon2 = radians(16.9251681)

dlon = lon2 - lon1
dlat = lat2 - lat1

a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
c = 2 * atan2(sqrt(a), sqrt(1 - a))

distance = R * c

print("Result:", distance)
print("Should be:", 278.546, "km")

Result: 8354.032533618552
Should be: 278.546 km


In [36]:
import requests

url = "https://realtor.p.rapidapi.com/properties/v2/detail"

querystring = {"property_id":"M2213622713"}

headers = {
    'x-rapidapi-host': "realtor.p.rapidapi.com",
    'x-rapidapi-key': "bdf6bde97amsh2f0d8da5849d952p1e4163jsn48237c3a4f33"
    }

response = requests.request("GET", url, headers=headers, params=querystring).json()

# print(response.text)
print(len(response))

2


In [63]:
import requests

url = "https://realtor.p.rapidapi.com/properties/v2/list-similar-rental-homes"

querystring = {"postal_code":"80227","property_id":"M2213622713"}

headers = {
    'x-rapidapi-host': "realtor.p.rapidapi.com",
    'x-rapidapi-key': "bdf6bde97amsh2f0d8da5849d952p1e4163jsn48237c3a4f33"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"properties":[{"property_id":2732525311,"listing_id":608456700,"ldp_url":"/realestateandhomes-detail/7410-W-Warren-Cir_Lakewood_CO_80227_M27325-25311","full_address_display":"7410 W Warren Cir, Lakewood, CO 80227","photo_url":"https://ar.rdcpix.com/3645539296/257b2f09a43b4c9034589010aed88100c-f0xd-w480_h480_q80.jpg","price_display":"$1,369+","beds":"1 - 3","baths":"1 - 2","baths_full":null,"baths_half":null,"sqft_display":"670+","search_flags":{"is_price_reduced":false,"is_new_listing":false,"is_pending":false},"saved_resource_id":null,"lat":39.67697,"lon":"","long":null,"agent_id":null,"price_per_sqft":2.0432835820895523,"sqft":670,"lot_sqft":null,"price":1369,"year_built":null,"beds_display":null,"baths_display":null,"address_display":null,"city_state_zip":null,"searchable_url":null,"list_date":null,"compared_price":null,"compared_year_built":null,"compared_sqft":null,"compared_lot_size":null,"prop_status":"for_rent","listed_date_time":null,"address":"7410 W Warren Cir","city":"Lake

In [ ]:
'prop_type'
sqft
baths
community/noncommunity
neighborhood
photos
